In [109]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [110]:
import numpy as np
import pandas as pd
from os import path
from collections import OrderedDict
from tqdm import tqdm
from typing import Dict

import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Embedding, Flatten, Input, Lambda
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.backend as K

from sklearn.metrics import roc_auc_score

import matplotlib.pyplot as plt

%matplotlib inline

In [111]:
tf.__version__

'2.12.0'

In [112]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

In [113]:
DIR_DATA = 'data'
DIR_MODEL = 'models'

### Get data

In [114]:
txt_path = '/content/drive/MyDrive/Graduationproject/test.csv'
df_full = pd.read_csv(path.join(DIR_DATA, txt_path))
df_full.head(1)

,user,business,rating,date
0,0,1481,5,1479040537


### Build the references

I'm planning to use the `Embedding` layer, so I need to link real ids of the users and movies to the order ones.

In [115]:
sorted(df_full.business.unique())[-10:]

[14517, 14519, 14524, 14528, 14536, 14543, 14547, 14566, 14567, 14568]

In [116]:
sorted(df_full.user.unique())[:10]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [117]:
unique_users = df_full.user.unique()
user_ids = dict(zip(unique_users, np.arange(unique_users.shape[0], dtype=np.int32)))

unique_businesss = df_full.business.unique()
business_ids = dict(zip(unique_businesss, np.arange(unique_businesss.shape[0], dtype=np.int32)))

In [118]:
df_full['user_id'] = df_full.user.apply(lambda u: user_ids[u])
df_full['business_id'] = df_full.business.apply(lambda m: business_ids[m])

df_full.head(1)

,user,business,rating,date,user_id,business_id
0,0,1481,5,1479040537,0,0


### Train/test split

Here the main idea is to extract some movies for users who have a big amount of positive reviews into the test subtest. I extract 2 movies for each user who have more than 20 positive reviews. This test subset won't be used during training, but these movies should appear in the top recommendations for each user accordingly.

#### Test subset

In [119]:
tmp_test = df_full[df_full.rating > 4]
tmp_test = tmp_test.groupby('user').business.count().reset_index()
tmp_test.shape

(623, 2)

In [120]:
conditions = (df_full.user.isin(tmp_test[tmp_test.business > 20].user)) & (df_full.rating > 4)
df_test = df_full[conditions].groupby('user').head(2).reset_index()

del df_test['index']
df_test.shape

(2, 6)

In [121]:
ground_truth_test = df_test.groupby('user_id').business_id.agg(list).reset_index()

ground_truth_test.head(1)

,user_id,business_id
0,253,"[498, 93]"


#### Training subset

In [122]:
df_train = pd.concat([df_full, df_test]).drop_duplicates(keep=False)
df_train.shape

(2828, 6)

In [123]:
# The assumption is that the recommendations should as many as possible high ranked movies
# that a specific user has already watched.

ground_truth_train = df_train[df_train.rating > 3].groupby('user_id').business_id.agg(list).reset_index()

ground_truth_train.head(1)

,user_id,business_id
0,0,"[0, 1]"


### Building triplets

Bayers Personalized Ranking requires for the training a triplet of the user, positive item and negative item. For each user, I create a pair of each positive ranked movie (the rank is higher than 3) with all negative movies (the rank is equal  3 and lower than).

In [124]:
df_triplest = pd.DataFrame(columns=['user_id', 'positive_m_id', 'negative_m_id'])
df_triplest

,user_id,positive_m_id,negative_m_id


In [125]:
%%time

data = []
users_without_data = []

for user_id in tqdm(df_train.user_id.unique()):
    positive_businesss = df_train[(df_train.user_id == user_id) & (df_train.rating > 3)].business_id.values
    negative_businesss = df_train[(df_train.user_id == user_id) & (df_train.rating <= 3)].business_id.values

    if negative_businesss.shape[0] == 0 or positive_businesss.shape[0] == 0:
        users_without_data.append(user_id)
        continue


    for positive_business in positive_businesss:
        for negative_business in negative_businesss:
            data.append({'user_id': user_id, 'positive_m_id': positive_business, 'negative_m_id': negative_business})

df_triplest = df_triplest.append(data, ignore_index=True)
# df_triplest.to_csv(path.join(DIR_DATA, 'triplets.csv'), index=False)

100%|██████████| 1001/1001 [00:01<00:00, 546.03it/s]
<timed exec>:17: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.


CPU times: user 1.95 s, sys: 10.6 ms, total: 1.96 s
Wall time: 2.06 s


In [126]:
df_triplest.shape, df_train.shape

((6464, 3), (2828, 6))

### BPR NN

In [127]:
num_users = unique_users.shape[0]
num_items = unique_businesss.shape[0]

num_users, num_items

(1001, 1825)

In [128]:
unique_business_ids = list(df_full.business_id.unique())

### Build a model

In [129]:
def bpr_predict(model: Model, user_id: int, item_ids: list, user_layer='user_embedding', item_layer='item_embedding'):
    """
    Predict by multiplication user vector by item matrix

    :return: list of the scores
    """
    user_vector = model.get_layer(user_layer).get_weights()[0][user_id]
    item_matrix = model.get_layer(item_layer).get_weights()[0][item_ids]

    scores = (np.dot(user_vector, item_matrix.T))

    return scores

In [130]:
@tf.function
def identity_loss(_, y_pred):
    return tf.math.reduce_mean(y_pred)

In [131]:
@tf.function
def bpr_triplet_loss(X: dict):
    """
    Calculate triplet loss - as higher the difference between positive interactions
    and negative interactions as better

    :param X: X contains the user input, positive item input, negative item input
    :return:
    """
    positive_item_latent, negative_item_latent, user_latent = X

    positive_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, positive_item_latent), axis=-1, keepdims=True)
    negative_interactions = tf.math.reduce_sum(tf.math.multiply(user_latent, negative_item_latent), axis=-1, keepdims=True)

    return tf.math.subtract(tf.constant(1.0), tf.sigmoid(tf.math.subtract(positive_interactions, negative_interactions)))

In [132]:
def out_shape(shapes):
    return shapes[0]


def build_model(num_users: int, num_items: int, latent_dim: int) -> Model:
    """
    Build a model for Bayesian personalized ranking

    :param num_users: a number of the unique users
    :param num_items: a number of the unique movies
    :param latent_dim: vector length for the latent representation
    :return: Model
    """
    user_input = Input((1,), name='user_input')

    positive_item_input = Input((1,), name='positive_item_input')
    negative_item_input = Input((1,), name='negative_item_input')
    # One embedding layer is shared between positive and negative items
    item_embedding_layer = Embedding(num_items, latent_dim, name='item_embedding', input_length=1)

    positive_item_embedding = Flatten()(item_embedding_layer(positive_item_input))
    negative_item_embedding = Flatten()(item_embedding_layer(negative_item_input))

    user_embedding = Embedding(num_users, latent_dim, name='user_embedding', input_length=1)(user_input)
    user_embedding = Flatten()(user_embedding)

    triplet_loss = Lambda(bpr_triplet_loss, output_shape=out_shape)([positive_item_embedding,
                                                             negative_item_embedding,
                                                             user_embedding])

    model = Model(inputs=[positive_item_input, negative_item_input, user_input], outputs=triplet_loss)

    return model

In [133]:
latent_dim = 350
batch_size = 256
num_epochs = 1
lr = 0.001

model = build_model(num_users, num_items, latent_dim)
model.compile(loss=identity_loss, optimizer=Adam(learning_rate=lr))

In [134]:
trainable_count = np.sum([K.count_params(w) for w in model.trainable_weights])
non_trainable_count = np.sum([K.count_params(w) for w in model.non_trainable_weights])

print('Total number of parameters: {:,}'.format(trainable_count + non_trainable_count))
print('Trainable number of parameters: {:,}'.format(trainable_count))
print('Non-trainable number of parameters: {:,}'.format(non_trainable_count))

print('Training data length: {:,}'.format(df_triplest.shape[0]))

Total number of parameters: 989,100.0
Trainable number of parameters: 989,100
Non-trainable number of parameters: 0.0
Training data length: 6,464


In [135]:
%%time

# 사용자 ID를 정수형으로 변환하여 텐서로 변환
user_input_tensor = tf.convert_to_tensor(df_triplest.user_id.astype(np.int32))

# 양의 아이템 ID를 정수형으로 변환하여 텐서로 변환
positive_item_input_tensor = tf.convert_to_tensor(df_triplest.positive_m_id.astype(np.int32))

# 음의 아이템 ID를 정수형으로 변환하여 텐서로 변환
negative_item_input_tensor = tf.convert_to_tensor(df_triplest.negative_m_id.astype(np.int32))

X = {
    'user_input': user_input_tensor,
    'positive_item_input': positive_item_input_tensor,
    'negative_item_input': negative_item_input_tensor
}


model.fit(X,
          tf.ones(df_triplest.shape[0]),
          batch_size=batch_size,
          epochs=num_epochs)

# model.save(path.join(DIR_MODEL, 'model.h5'))

26/26 [==============================] - 2s 26ms/step - loss: 0.4627
CPU times: user 1.71 s, sys: 90.4 ms, total: 1.8 s
Wall time: 2.48 s


### Evaluation

In [136]:
def full_auc(model: Model, ground_truth: Dict[int, list], items: list) -> float:
    """
    Measure AUC for model and ground truth for all items

    :param model:
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :return: AUC
    """

    number_of_items = len(items)
    scores = []

    for user_id, true_item_ids in ground_truth:
        predictions = bpr_predict(model, user_id, items)
        grnd = np.zeros(number_of_items, dtype=np.int32)

        for p in true_item_ids:
            index = items.index(p)
            grnd[index] = 1

        if true_item_ids:
            scores.append(roc_auc_score(grnd, predictions))

    return sum(scores) / len(scores)

In [137]:
def mean_average_precision_k(model: Model,
                           ground_truth: Dict[int, list],
                           items: list,
                           k=100) -> float:
    """
    Calculate mean eavarage precission per user

    :param model:
    :param ground_truth: dictionary of the users and the high ranked movies for the specific user
    :param items: a list of the all available movies
    :param k: top N recommendations per user
    :return: mean eavarage precission
    """
    scores = []

    for user, actual in ground_truth:
        predictions = bpr_predict(model, user, items)
        predictions = dict(zip(items, predictions))
        predictions = sorted(predictions.items(), key=lambda kv: kv[1], reverse=True)[:k]
        predictions = list(OrderedDict(predictions).keys())

        score = 0.0
        num_hits = 0.0

        for i, p in enumerate(predictions):
            if p in actual:
                num_hits += 1.0
                score += num_hits / (i + 1.0)

        score = score / min(len(actual), k)
        scores.append(score)

    return np.mean(scores)

#### Train

In [138]:
print(f'AUC train: {full_auc(model, ground_truth_train.values, unique_business_ids)}')

AUC train: 0.6253819281617754


In [139]:
print(f'Mean average precision train: {mean_average_precision_k(model, ground_truth_train.values, unique_business_ids)}')

Mean average precision train: 0.1550004608182973


#### Test

In [140]:
print(f'AUC test: {full_auc(model, ground_truth_test.values, unique_business_ids)}')

AUC test: 0.46982995063082833


In [141]:
print(f'Mean average precision test: {mean_average_precision_k(model, ground_truth_test.values, unique_business_ids)}')

Mean average precision test: 0.0
